## Retriever

Retriever는 주어진 쿼리(질문)에 대해 관련성이 높은 문서나 정보를 데이터베이스나 문서 컬렉션에서 찾아내는 시스템.

#### 주요 기능
1. 의미 기반 검색(Semantic Search)
    - 키워드 매칭이 아닌 의미적 유사도를 기반으로 문서 검색
    - 임베딩 벡터 간의 유사도를 계산하여 관련 문서 추출

2. 효율적인 검색
    - 대량의 문서에서 빠르게 관련 정보를 찾아냄
    - 벡터 데이터베이스(FAISS, Pinecone, Chroma 등) 활용

3. 컨텍스트 제공    - 
LLM에게 답변 생성을 위한 관련 컨텍스트 제공
    - 환각(hallucination) 감소 및 정확도 향상

#### Retriever의 종류
1. Dense Retriever
    - 신경망 기반 임베딩 사용
    - 예: Sentence Transformers, OpenAI Embeddings

2. Sparse Retriever
    - 키워드 기반 검색
    - 예: BM25, TF-IDF

3. Hybrid Retriever
    - Dense + Sparse 방식 결합
    - 더 나은 검색 성능 제공

#### RAG에서의 역할
사용자 질문 → Retriever → 관련 문서 검색 → LLM → 답변 생성

> Retriever는 외부 지식을 활용하여 LLM이 더 정확하고 최신의 정보를 기반으로 답변할 수 있도록 돕는다.

In [1]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일에서 환경 변수 로드

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
# RetrievalQA : 문서 검색 및 질문 응답 체인
# - 문서 검색 기능과 질문 응답 기능을 결합한 체인
from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI # OpenAI 챗 모델
from langchain_community.vectorstores import FAISS

# 모델 초기화  
model = ChatOpenAI(
    model_name="gpt-4o-mini", 
    temperature=0 # 창의성 제어, 0은 가장 결정적인 응답
    )

vector_store = FAISS.load_local(
    "./db/faiss_vector_store",  # 저장된 경로
    embedding_model, # 임베딩 모델
    allow_dangerous_deserialization=True # FAISS 벡터 스토어 로드 시 필요(보안 경고 무시)
    )

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# 문서 검색기 설정 (예: 벡터 데이터베이스에서 검색)
retrieval_qa = RetrievalQA.from_chain_type(
    llm=model,

    # 문서 검색기 설정
    retriever=retriever,

    # 문서 결합 방식 지정("stuff", "map_reduce" 등),  
    # stuff : retriever로 검색된 모든 문서를 하나로 합쳐 처리
    chain_type="stuff" 
)

retrieval_qa


RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000017B36570550>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000017B36571090>, root_client=<openai.OpenAI object at 

#### Tom Sawyer라는 책에 대한 질의 응답

In [9]:
# Tom Sawyer 책의 전체 줄거리 요약 질문
retrieval_qa.invoke('Tom Sawyer 책의 주요 줄거리와 내용을 요약해줘')

{'query': 'Tom Sawyer 책의 주요 줄거리와 내용을 요약해줘',
 'result': '"톰 소여의 모험"은 마크 트웨인이 쓴 소설로, 19세기 미국의 작은 마을에서 자라는 소년 톰 소여의 이야기를 다룹니다. 톰은 모험을 사랑하는 소년으로, 친구인 허크 핀과 함께 다양한 모험을 경험합니다. \n\n소설은 톰이 보물 찾기를 제안하며 시작되며, 그는 허크와 함께 여러 가지 사건에 휘말리게 됩니다. 톰은 묘지에서의 모험과 동굴에서의 사건을 겪으며, 인종 조와 같은 위협적인 인물과 마주하게 됩니다. 또한, 톰은 친구 베키와의 관계에서도 갈등을 겪고, 친구 머프 포터의 재판에 참여하여 그를 도우려는 결심을 하게 됩니다.\n\n이 이야기는 우정, 용기, 그리고 도덕적 선택에 대한 주제를 탐구하며, 톰의 성장과 모험을 통해 독자에게 재미와 교훈을 제공합니다.'}

In [7]:
retrieval_qa.invoke('마을 무덤에 있떤 남자를 누가 죽였나요?')

{'query': '마을 무덤에 있떤 남자를 누가 죽였나요?', 'result': 'Injun Joe가 의사를 죽였습니다.'}

In [ ]:
retrieval_qa.invoke('인디언 조가 누구를 죽였나요?') # 질문에 대한 응답

{'query': '인디언 조가 누구를 죽였나요?', 'result': '인디언 조는 의사를 죽였습니다.'}

In [6]:
retrieval_qa.invoke('인디언 조는 의사를 어떻게 죽였나요?')

{'query': '인디언 조는 의사를 어떻게 죽였나요?', 'result': '인디언 조는 의사를 칼로 죽였습니다.'}